In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [15]:
## 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
## 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
## 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
## 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
## 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [16]:
# region = 'daqing'
# sample_region = 'DAQING'
# region = 'tianjin'
# sample_region = 'TIANJIN'
# region = 'wuhan'
# sample_region = 'WUHAN'
# region = 'prd'
# sample_region = 'PRD'
region = 'tibet'
sample_region = 'TIBET'

In [17]:
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

## 指数的计算
# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(img):
    image = ee.Image(img).clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

In [18]:
# TIME = [['2000'],['2005'],['2010'],['2015'],['2020']]
TIME = [['2000'],['2015'],['2020']]
# TIME = [['2020']]
for time in TIME:
    print(time[0])
    ## 导入人工采集的样本
    manual_samples1 = ee.FeatureCollection('users/311605001111/manual_samples/' + sample_region +'_' + time[0] + 'merge')
    # print(manual_samples1.size().getInfo())
    manual_samples2 = ee.FeatureCollection('users/311605001111/manual_samples/' + region +'_supply_' + time[0])
    manual_samples = ee.FeatureCollection([manual_samples1,manual_samples2]).flatten()
    print('manual-sample number:{}'.format(manual_samples.size().getInfo()))
    
    dataset_id = sample_region + '_' + time[0]
    assetID = 'users/311605001111/manual_samples/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': manual_samples,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

2000
manual-sample number:1279
2015
manual-sample number:2258
2020
manual-sample number:2424


# 循环

In [ ]:
def image_accuracy(img):
    image = maskSR(water_index(img)).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
    manual_point = manual_samples.filter(ee.Filter.eq('Image_id',img.get('system:id'))).map(lambda i : i.centroid())
    inter_points = image.sampleRegions(**{
        'collection': manual_point,
        'properties': ['label'],
        'scale': 30,
        'geometries': True,
    })
    inter_samples = inter_points.map(lambda i : i.setMulti({'Image_id':img.get('system:id')}))
    return inter_samples

# def removeProperty(feat):
#     selectProperties = ee.Feature(feat).propertyNames().filter(ee.Filter.neq('item', 'random')).filter(ee.Filter.neq('item', 'waterclass'))  
#     return feat.select(selectProperties)
def removeProperty(feat):
    selectProperties = ee.Feature(feat).propertyNames().filter(ee.Filter.neq('item', 'random'))
    return feat.select(selectProperties)

In [ ]:
# TIME = [['2000','0105'],['2005','0105'],['2010','0610'],['2015','1115'],['2020','1620']]
TIME = [['2000'],['2005'],['2010'],['2015'],['2020']]
# TIME = [['2000'],['2015'],['2020']]
# TIME = [['2020']]
for time in TIME:
    print(time[0])
    ## 导入人工采集的样本
    # manual_samples1 = ee.FeatureCollection('users/311605001111/manual_samples/' + sample_region +'_' + time[0] + 'merge')
    # print(manual_samples1.size().getInfo())
    # manual_samples2 = ee.FeatureCollection('users/311605001111/manual_samples/' + region +'_supply_' + time[0])
    # manual_samples = ee.FeatureCollection([manual_samples1,manual_samples2]).flatten()
    
    manual_samples = ee.FeatureCollection('users/311605001111/manual_samples/' + sample_region +'_' + time[0] + 'merge')
    print('manual-sample number:{}'.format(manual_samples.size().getInfo()))

    # 图像集，image_col
    image_id = ee.List(manual_samples.distinct('Image_id').aggregate_array('Image_id'))
    image_set = ee.List([])
    for i in image_id.getInfo():
        if 'LT05' in i:
            image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
        elif 'LE07' in i:
            image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
        else:
            image_set = image_set.add(ee.Image(str(i)).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    image_col = ee.ImageCollection(image_set)
    ## 导入自动采集的样本
    auto_samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_9920_v2')
    print('auto-sample number:{}'.format(auto_samples.size().getInfo()))
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    trainedClassifier = ee.Classifier.smileRandomForest(150).train(auto_samples,label,bands)
    samples_dataset = image_col.map(image_accuracy).flatten()
    # 利用误差矩阵进行验证
    test_accuracy = samples_dataset.errorMatrix('label', 'waterclass')
    print(test_accuracy.getInfo())
    print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
    print('kappa:{}'.format(test_accuracy.kappa().getInfo()))    
    
#     water = samples_dataset.filter(ee.Filter.eq('label',1)).randomColumn('random').sort('random').limit(200)
#     no_water = samples_dataset.filter(ee.Filter.eq('label',0)).randomColumn('random').sort('random').limit(800)
#     com_SamplePoints = ee.FeatureCollection([water,no_water]).flatten()
#     test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
#     print(test_accuracy.getInfo())
#     print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
#     print('kappa:{}'.format(test_accuracy.kappa().getInfo()))


In [ ]:
## 情况一
# TIME = [['2000','0105'],['2005','0105'],['2010','0610'],['2015','1115'],['2020','1620']]
# TIME = [['2000'],['2005'],['2010'],['2015'],['2020']]
TIME = ['2000','2005','2010','2015','2020']
for time in TIME:
    print(time)
    ## 导入人工采集的样本
    manual_samples = ee.FeatureCollection('users/311605001111/manual_samples/' + sample_region +'_' + time + 'merge')
    print('manual-sample number:{}'.format(manual_samples.size().getInfo()))
    # 图像集，image_col
    image_id = ee.List(manual_samples.distinct('Image_id').aggregate_array('Image_id'))
    image_set = ee.List([])
    for i in image_id.getInfo():
        if 'LT05' in i:
            image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
        elif 'LE07' in i:
            image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
        else:
            image_set = image_set.add(ee.Image(str(i)).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    image_col = ee.ImageCollection(image_set)
    ## 导入自动采集的样本
    auto_samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_9920_v2')
    print('auto-sample number:{}'.format(auto_samples.size().getInfo()))
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    trainedClassifier = ee.Classifier.smileRandomForest(150).train(auto_samples,label,bands)
    samples_dataset = image_col.map(image_accuracy).flatten()
    # 利用误差矩阵进行验证
    water = samples_dataset.filter(ee.Filter.eq('label',1)).randomColumn('random').sort('random').limit(200)
    no_water = samples_dataset.filter(ee.Filter.eq('label',0)).randomColumn('random').sort('random').limit(800)
    com_SamplePoints = ee.FeatureCollection([water,no_water]).flatten()
    test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
    print(test_accuracy.getInfo())
    print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
    print('kappa:{}'.format(test_accuracy.kappa().getInfo()))
    dataset_id = sample_region + time
    assetID = 'users/311605001111/man_samples_1000' + '/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': com_SamplePoints.map(removeProperty),
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()


In [ ]:
## 情况二
# TIME = [['2000','0105'],['2005','0105'],['2010','0610'],['2015','1115'],['2020','1620']]
# TIME = ['2000','2005','2010','2015','2020']
TIME = ['2005']
for time in TIME:
    print(time)
    ## 导入人工采集的样本
    manual_samples = ee.FeatureCollection('users/311605001111/manual_samples/' + sample_region +'_' + time + 'merge')
    print('manual-sample number:{}'.format(manual_samples.size().getInfo()))
    # 图像集，image_col
    image_id = ee.List(manual_samples.distinct('Image_id').aggregate_array('Image_id'))
    image_set = ee.List([])
    for i in image_id.getInfo():
        if 'LT05' in i:
            image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
        elif 'LE07' in i:
            image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
        else:
            image_set = image_set.add(ee.Image(str(i)).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    image_col = ee.ImageCollection(image_set)
    ## 导入自动采集的样本
    auto_samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_9920_v2')
    print('auto-sample number:{}'.format(auto_samples.size().getInfo()))
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    trainedClassifier = ee.Classifier.smileRandomForest(150).train(auto_samples,label,bands)
    samples_dataset = image_col.map(image_accuracy).flatten()
    # 利用误差矩阵进行验证
    diff_water = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0))
    diff_nowater = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
    cor_water = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',1)).randomColumn('random').sort('random').limit(200-diff_water.size().getInfo())
    cor_nowater = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',0)).randomColumn('random').sort('random').limit(800-diff_nowater.size().getInfo())
    total_samples = ee.FeatureCollection([diff_water,diff_nowater,cor_water,cor_nowater]).flatten()
    test_accuracy = total_samples.errorMatrix('label', 'waterclass')
    print(test_accuracy.getInfo())
    print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
    print('kappa:{}'.format(test_accuracy.kappa().getInfo()))
    dataset_id = sample_region + time
    assetID = 'users/311605001111/man_samples_1000' + '/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': total_samples.map(removeProperty),
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

# 异常点

In [ ]:
time = '2005'

manual_samples = ee.FeatureCollection('users/311605001111/manual_samples/' + sample_region +'_' + time + 'merge')
print('manual-sample number:{}'.format(manual_samples.size().getInfo()))
# 图像集，image_col
image_id = ee.List(manual_samples.distinct('Image_id').aggregate_array('Image_id'))
image_set = ee.List([])
for i in image_id.getInfo():
    if 'LT05' in i:
        image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    elif 'LE07' in i:
        image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    else:
        image_set = image_set.add(ee.Image(str(i)).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
image_col = ee.ImageCollection(image_set)
## 导入自动采集的样本
auto_samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_9920_v2')
print('auto-sample number:{}'.format(auto_samples.size().getInfo()))
label = 'waterclass'
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
trainedClassifier = ee.Classifier.smileRandomForest(150).train(auto_samples,label,bands)
samples_dataset = image_col.map(image_accuracy).flatten()
# 利用误差矩阵进行验证
water = samples_dataset.filter(ee.Filter.eq('label',1)).randomColumn('random',1,'uniform').sort('random').limit(200)
no_water = samples_dataset.filter(ee.Filter.eq('label',0)).randomColumn('random',1,'uniform').sort('random').limit(800)
com_SamplePoints = ee.FeatureCollection([water,no_water]).flatten()
test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))
test_accuracy = samples_dataset.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

In [ ]:
manual_samples = ee.FeatureCollection('users/311605001111/manual_samples/' + sample_region +'_' + time + 'merge')
id_col = ee.List(manual_samples.distinct('Image_id').aggregate_array('Image_id'))
print(id_col.getInfo())
for i in id_col.getInfo():
    print(i)
    diff_water = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0))
    diff_nowater = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
    all_outlier = ee.FeatureCollection([diff_water,diff_nowater]).flatten()
    outlier = all_outlier.filter(ee.Filter.eq('Image_id',i))
    print(outlier.size().getInfo())

In [ ]:
diff_water = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0))
diff_nowater = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
all_outlier = ee.FeatureCollection([diff_water,diff_nowater]).flatten()
outlier = all_outlier.filter(ee.Filter.eq('Image_id','LANDSAT/LT05/C01/T1_SR/LT05_119028_20050713')).map(lambda i : i.centroid())
print(outlier.size().getInfo())
print(outlier.getInfo())

In [ ]:
task = ee.batch.Export.table.toDrive(**{
    'collection': ee.FeatureCollection(outlier),
    'description':'outlier',
    'folder':'qq',
    'fileFormat': 'shp',
    'selectors':['label','Image_id']
})
task.start()

In [ ]:
# LC08_122044_20201202
[113.44828144105588, 23.11162081260809]
[113.81352665544533, 22.96499602660751]
[113.66542356355623, 22.86844354358508] 
[112.88283089228791, 22.409063926472744]
[113.76768604046451, 23.154266767565872]
[113.04889464681561, 23.170678206544128]

# LC08_122044_20200828
[112.89919260995767, 22.433796784823013]
[113.76768604046451, 23.154266767565872]
[112.9445469331655, 22.467933444907203]
[113.21129997788758, 22.416550451733233]
[113.43795458267364, 22.49995940023891]
[112.97702456228362, 23.0245051963048]

# LE07_122044_20200226
[113.44798873260294, 23.111625356385463]
[113.44798873260294, 23.111625356385463]
[113.91624732089959, 22.376117488935375] 
[113.44798873260294, 23.111625356385463]
[113.88793431073869, 22.54125771572865]


# 分解

##  RF训练

In [ ]:
year = '2010'
year_time = '0610'

In [ ]:
### 将人工选取样本的影像变成影像集
## 导入人工采集的样本
manual_samples = ee.FeatureCollection('users/311605001111/manual_samples/' + sample_region +'_' + year + 'merge')
print('manual-sample number:{}'.format(manual_samples.size().getInfo()))
# 水体的样本点数
samplepoint_water = manual_samples.filter(ee.Filter.eq('label',1))
print('water sample number:{}'.format(samplepoint_water.size().getInfo()))
# 非水体的样本点数
samplepoint_land = manual_samples.filter(ee.Filter.eq('label',0))
print('nowater sample number:{}'.format(samplepoint_land.size().getInfo()))


image_id = ee.List(manual_samples.distinct('Image_id').aggregate_array('Image_id'))
image_set = ee.List([])
for i in image_id.getInfo():
    if 'LT05' in i:
        image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    elif 'LE07' in i:
        image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    else:
        image_set = image_set.add(ee.Image(str(i)).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
# 图像集，image_col
image_col = ee.ImageCollection(image_set)
print(image_col.aggregate_array('system:id').getInfo())

In [ ]:
## 导入自动采集的样本
samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_' + year_time)
print('auto-sample number:{}'.format(samples.size().getInfo()))

label = 'waterclass'
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
auto_samples = samples
trainedClassifier = ee.Classifier.smileRandomForest(150).train(auto_samples,label,bands)

##  精度验证函数

In [ ]:
def image_accuracy(img):
    image = maskSR(water_index(img)).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
    manual_point = manual_samples.filter(ee.Filter.eq('Image_id',img.get('system:id'))).map(lambda i : i.centroid())
    inter_points = image.sampleRegions(**{
        'collection': manual_point,
        'properties': ['label'],
        'scale': 30,
        'geometries': True,
    })
    inter_samples = inter_points.map(lambda i : i.setMulti({'Image_id':img.get('system:id')}))
    return inter_samples

samples_dataset = image_col.map(image_accuracy).flatten()
print(samples_dataset.size().getInfo())

In [ ]:
# 利用误差矩阵进行验证
water = samples_dataset.filter(ee.Filter.eq('label',1)).randomColumn('random').sort('random').limit(200)
no_water = samples_dataset.filter(ee.Filter.eq('label',0)).randomColumn('random').sort('random').limit(800)
com_SamplePoints = ee.FeatureCollection([water,no_water]).flatten()
test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

test_accuracy = samples_dataset.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

In [ ]:
# 利用误差矩阵进行验证
diff_water = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0))
diff_nowater = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
cor_water = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',1)).randomColumn('random').sort('random').limit(200-diff_water.size().getInfo())
cor_nowater = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',0)).randomColumn('random').sort('random').limit(800-diff_nowater.size().getInfo())
total_samples = ee.FeatureCollection([diff_water,diff_nowater,cor_water,cor_nowater]).flatten()

test_accuracy = total_samples.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

# 附录

In [ ]:
a = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
id = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1)).distinct('Image_id').aggregate_array('Image_id')

for i in range(0,id.size().getInfo(),1):
    print(id.get(i).getInfo())
    print('total:',a.filter(ee.Filter.eq('Image_id',id.get(i))).size().getInfo())

In [ ]:
a = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0))
id = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0)).distinct('Image_id').aggregate_array('Image_id')

for i in range(0,id.size().getInfo(),1):
    print(id.get(i).getInfo())
    print('total:',a.filter(ee.Filter.eq('Image_id',id.get(i))).size().getInfo())

In [ ]:
q = a.filter(ee.Filter.eq('Image_id','LANDSAT/LT05/C01/T1_SR/LT05_122039_20051107'))
print(q.size().getInfo())

In [ ]:
# shp格式
selector = ['Image_id','label','waterclass']
geemap.ee_export_vector_to_drive(q, description='LT05_122039_20051107_cor', folder='corPoints3', file_format='shp', selectors=selector)

In [ ]:
print(samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1)).size().getInfo())

print(samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1)).distinct('Image_id').aggregate_array('Image_id').getInfo())

# print(samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0)).first().getInfo())

In [ ]:
a = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0))
id = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0)).distinct('Image_id').aggregate_array('Image_id')

for i in range(0,id.size().getInfo(),1):
    print(id.get(i).getInfo())
    print('total:',a.filter(ee.Filter.eq('Image_id',id.get(i))).size().getInfo())

In [ ]:
for i in range(0,image_id.size().getInfo(),1):
    print(com_SamplePoints.filter(ee.Filter.eq('Image_id',image_id.get(i))).size().getInfo())
    print(image_id.get(i).getInfo())
    print('water:',com_SamplePoints.filter(ee.Filter.eq('Image_id',image_id.get(i))).filter(ee.Filter.eq('label',1)).size().getInfo())
    print('no-water:',com_SamplePoints.filter(ee.Filter.eq('Image_id',image_id.get(i))).filter(ee.Filter.eq('label',0)).size().getInfo())

In [ ]:
print(samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0)).size().getInfo())

print(samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0)).distinct('Image_id').aggregate_array('Image_id').getInfo())

print(samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0)).first().getInfo())

# 样本分布

In [ ]:
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

# 图像可视化参数
visParams = {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4}

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2020-01-01', '2020-12-31')
image = l8.map(maskSR).median().clip(roi)
refer_img = image.visualize(**{'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4})
Map.addLayer(image, visParams,'image median')

task = ee.batch.Export.image.toAsset(**{
    'image': refer_img,
    'description': region,
    'assetId': 'users/311605001111/basemap_image/' + region,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

In [ ]:
# region = 'daqing'
# sample_region = 'DAQING'
# region = 'tianjin'
# sample_region = 'TIANJIN'
# region = 'wuhan'
# sample_region = 'WUHAN'
region = 'prd'
sample_region = 'PRD'
# region = 'tibet'
# sample_region = 'TIBET'

In [ ]:
# 合成影像
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
# # 大庆市
# re = [125.3055, 45.8362, 123.9355, 46.8762]
# # 天津市
# re = [117.4944, 38.6272, 115.9244, 39.6372]
# # 武汉市
# re = [115.0993, 29.8642, 113.7393, 30.9242]
# # 珠三角
re = [114.0514, 22.2347, 112.7614, 23.2547]
# 青藏高原
# re = [91.3626, 34.6579, 89.8826, 35.8279]

fig = plt.figure(figsize=(12, 8))
refer_img = ee.Image('users/311605001111/basemap_image/' + region).visualize(**{'opacity':0.85})
# refer_img = image.visualize(**{'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4,'opacity':0.7})
manual_samples = ee.FeatureCollection('users/311605001111/man_samples_1000/' + sample_region + '2020')
print('manual-sample number:{}'.format(manual_samples.size().getInfo()))
nowater = manual_samples.filter(ee.Filter.eq('label',0)).draw('red')
water = manual_samples.filter(ee.Filter.eq('label',1)).draw('blue')
blend = refer_img.blend(nowater).blend(water)
Map.addLayer(blend, {}, "Blend")

ax = cartoee.get_map(blend, region=re)
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = '(d)        Region D', fontsize=28)